In [7]:
import os
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt 
import PIL

# Model-Related
import tensorflow as tf
from tensorflow.keras import layers

# Data Preprocessing
from sklearn.model_selection import train_test_split

### Emotion Recognition in Visual Data

This module Extracts / Preprocesses & Explores data from the following datasets:
- CK+
- RAVDESS Facial Landmark Tracking
- Yale Face Database
- AffWild 2
- Microsoft FER+

It will have a Convolutional Neural Network implemented with Keras + TensorFlow back-end and trained on this data.

We will be creating testing sets of the three team members capturing themselves.


### CK+ Dataset

In [8]:
def getCK():
    ck_data = h5py.File('../Datasets/CK_data.h5', 'r')
    
    X = np.array(ck_data.get('data_pixel'))
    y = np.array(ck_data.get('data_label'))
    
    return X, y

def splitTrainValidate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9)
    
    return X_train, y_train, X_test, y_test



### FER+ Dataset

In [9]:
from io import StringIO

def parsePixels(pixelString):
    data = pixelString.split(" ")
    
    i = 0
    for each in data:
        data[i] = np.uint8(data[i])
        i += 1    
        
    return data


def getFER():
    fer_data = pd.read_csv('../Datasets/fer2013.csv')
    fer_data['pixels'] = fer_data['pixels'].apply(parsePixels)
    
    fer_training = fer_data[fer_data['Usage'] == 'Training']    
    fer_validation_private = fer_data[fer_data['Usage'] == 'PrivateTest']
    fer_validation_public = fer_data[fer_data['Usage'] == 'PublicTest']
    
    
#     # Save Training Images
#     for idx, each in enumerate(fer_training['pixels']):
#         fer_training['pixels'].values[idx] =  np.asarray(fer_training['pixels'][idx]).reshape(48, 48)
#         img = PIL.Image.fromarray(fer_training['pixels'][idx])
#         img.save('../Datasets/FER2013/FER_TRAIN/' + 'fer_train_' + str(idx) + '.png')
        
#     # Save Training Labels
#     fer_training_y = fer_training['emotion']

    
    
#     # Save Validation Images (Private)    
#     for idx, each in enumerate(fer_validation_private['pixels']):
#         fer_validation_private['pixels'].iloc[idx] = np.asarray(fer_validation_private['pixels'].iloc[idx]).reshape(48, 48)
#         img = PIL.Image.fromarray(fer_validation_private['pixels'].iloc[idx])
#         img.save('../Datasets/FER2013/FER_VAL_PRIVATE/' + 'fer_val_private_' + str(idx) + '.png')
        
#     # Save Validation Labels (Private)    
#     fer_validation_private_y = fer_validation_private['emotion']    
        
    

#     # Save Validation Images (Public)
#     for idx, each in enumerate(fer_validation_public['pixels']):
#         fer_validation_public['pixels'].iloc[idx] = np.asarray(fer_validation_public['pixels'].iloc[idx]).reshape(48, 48)
#         img = PIL.Image.fromarray(fer_validation_public['pixels'].iloc[idx])
#         img.save('../Datasets/FER2013/FER_VAL_PUBLIC/' + 'fer_val_public' + str(idx) + '.png')
        
#     # Save Validation Labels (Public)    
#     fer_validation_public_y = fer_validation_public['emotion']    
            
    return fer_data


    
    

In [17]:
# Get CK+ Dataset
X_CK, y_CK = getCK()

# Get FER+ Dataset
fer_data = getFER()

In [49]:
X_CK[0].reshape(1, 48, 48)

(1, 48, 48)

In [43]:
X_all = np.zeros((1, 48, 48))
y_all = np.zeros((1, 48, 48))

i = 0
while i <= fer_data.shape[0]:
    X_all = np.append(X_all, fer_data['pixels'][i])
    y_all = np.append(y_all, fer_data['emotion'][i])
    i += 1
    

KeyboardInterrupt: 

In [38]:
X_all = X_all.reshape(-1, 48, 48)

In [39]:
X_train, y_train, X_test, y_test = splitTrainValidate(X_all, y_all)

print("Training Data: ", X_train.shape)
print("Training Labels", y_train.shape)
print("Validation Data: ", X_test.shape)
print("Validation Labels: ",y_test.shape)

Training Data:  (33181, 48, 48)
Training Labels (33181,)
Validation Data:  (3687, 48, 48)
Validation Labels:  (3687,)


### Image Classification with Convolutional Neural Network

- Using Keras with Tensorflow Back-end to Design, Train, Tune & Test a CNN

In [41]:
# Prepare inputs.
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_train = np.float32(X_train) / X_train.max()
y_train = tf.keras.utils.to_categorical(y_train)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)
X_test = np.float32(X_test) / X_test.max()
y_test = tf.keras.utils.to_categorical(y_test)

model = tf.keras.models.Sequential([

        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = "same"),
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = "same"),     
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2)),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.Conv2D(128, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = "same"),    
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.Conv2D(128, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = "same"),    
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2)),
        tf.keras.layers.Dropout(0.25),
    
        tf.keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = 'same'),
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = 'same'),
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.Conv2D(256, kernel_size = (1, 1), activation = 'relu', strides = (1, 1), padding = 'same'),
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2)),    
        tf.keras.layers.Dropout(0.25),
    
        tf.keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = 'same'),
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu', strides = (1, 1), padding = 'same'),
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.Conv2D(256, kernel_size = (1, 1), activation = 'relu', strides = (1, 1), padding = 'same'),
        tf.keras.layers.ZeroPadding2D((1,1)),
        tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2)),    
        tf.keras.layers.Dropout(0.25),
    
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation = 'relu'),
        tf.keras.layers.Dense(1024, activation = 'relu'),
    
        tf.keras.layers.Dense(7, activation = 'softmax')
])

model.compile(optimizer = tf.optimizers.Adam(0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 128, epochs = 3)

Epoch 1/3


ValueError: in user code:

    /home/ip/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/ip/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ip/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ip/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ip/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ip/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:789 run_step

In [ ]:
model.evaluate(X_test, y_test)

# Categorical Accuracy?

In [ ]:
pred = model.predict(X_test)

emotion_labels = ['Anger','Disgust','Fear','Happy','Sadness','Surprise', 'Contempt']

figure = plt.figure(figsize = (16, 9))

i = 0
imgcount = 1
plt.title('Incorrectly Classified Images.')

while imgcount < 11:
    
    if(y_test[i].argmax() != pred[i].argmax()):
        figure.add_subplot(2, 5, imgcount)
        plt.imshow(X_test[i].reshape((48, 48)))
        imgcount += 1
        plt.title('Y: '+ emotion_labels[pred[i].argmax()] +  ', Y_0: ' + emotion_labels[y_test[i].argmax()])
        
    i += 1
    
plt.show()